# Constraint programming

In constraint programming or constraint optimization the goal is to idendtify a feasible solution (or more), where the problem can be modeled in terms of arbitary constraints.

## Install OR-Tools

Installing OR-Tools is straight forward using conda:

```conda install -c conda-forge ortools-python```

or simply pip

```python -m pip install --upgrade --user ortools```

for more information see [Google OR-Tools Overview](https://developers.google.com/optimization/install)

## Example count enemy soldiers

We want to defeat the army of our opponent and therefore we like to know the the number of enemy soldiers. Lets denote the number of soldiers by $n$.

Because counting the soldiers one by one would be time consuming and error prone, we asked our scouts to simply count the number of soldiers in the first row while they were marching through different landscapes.

Hence the scouts reported:

- scout 1: "At the bridge there were 13 soldiers in first row"
- scout 2: "At the crossing there were 19 soldiers in the first row"
- scout 3: "In the open field there were 37 soldiers in the first row"

So lets translate this into math: Scout 1 says that the number of soldiers is divisible by 13 and scout 2 (respk. scout 3) says that the number is divisible by 19 (respk. 37). And this can written as [congruences](https://en.wikipedia.org/wiki/Modular_arithmetic)

$$
\begin{array}{lll}
n & \equiv 0 & \mod (13) \\
n & \equiv 0 & \mod (19) \\
n & \equiv 0 & \mod (37) 
\end{array}
$$

In addition we somehow know that the number of soldiers of the opponents army can not exceed 10000 soldiers.

*remark:* For simplicity of this example we assume that each row has the same number of soldiers. However if there would be less soldiers in the last row, then the left hand side of the above congruence would be excatly this number.


### background info

The example is well known since the 3 century and is a simple application of the [chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) and requires only some simple math to solve it directly.
But we like to point out that the use of congrunences in a LP for example is not that straight forward.

## general steps

1. instantiate the model and the solver
1. declare variables 
1. declare constraints
1. (optional) define objective
1. apply solver

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
data = {
    'army_ub': 10000,
    'counts': [13,19,37]
}

In [ ]:
data

{'army_ub': 10000, 'counts': [13, 19, 37]}

In [ ]:
def chinese_remainder(data):
    
    # instantiate the model and the solver
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    # declare variable (together with bounds)
    army = model.NewIntVar(1, data['army_ub'], 'army')
    # declare constraints
    for i in data['counts']:
        model.AddModuloEquality(0, army, i)
    # apply solver
    status = solver.Solve(model)
    
    if status == cp_model.OPTIMAL:
        print("solution time %s milliseconds" %round(solver.WallTime(), 3))
        print("soldiers in enemy army: %s" %solver.Value(army))
    else:
        print('solver could not find a feasible solution')
    return solver.Value(army)

In [ ]:
solution = chinese_remainder(data)

solution time 0.038 milliseconds
soldiers in enemy army: 9139


In [ ]:
# check solution, expected value = 0
j = 0
for i in data['counts']:
    j = j+1
    print("constraint %s: %d mod(%d) = %d" %(j, solution, i, solution % i) )

constraint 1: 9139 mod(13) = 0
constraint 2: 9139 mod(19) = 0
constraint 3: 9139 mod(37) = 0


In particular because the example is so simple (all remainder are equal to zero), we an solution is given by simply multiplying the divisors. (N.B. This would not be true, if there is one non zero remainder)

# Multiple solutions

In the previous setting there was only one solution due to the choice of the upper bound, i.e. maximal 10000 soldiers in teh army.

In [ ]:
data['army_ub'] = 100000

In [ ]:
chinese_remainder(data)

solution time 0.003 milliseconds
soldiers in enemy army: 9139


9139